# Imports and API setups

In [ ]:
from __future__ import print_function    # (at top of module)
import warnings
warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
import math
import seaborn as sns
import config


# Spotify API Setup
client_credentials_manager = SpotifyClientCredentials(config.client_id, config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Enables verbose JSON requests tracing
sp.trace=False

## Getting the data

In [ ]:
# My public user ID on Spotify
userId = "spotify:user:217unxkx4en4irnq4nkvgax6y"

# The id of the playlist where the data will come from
playlistID = "spotify:user:217unxkx4en4irnq4nkvgax6y:playlist:1v3tLpVLqty2FMPm4oY2rs"

# File name to write to
fileName = "mydata_new_stuff.csv"

# Columns for my pandas DataFrame in which we will keep the data
columns = ["song_id","song_title", "artist", "popularity", "energy", "liveness", "tempo"
          , "speechiness", "acousticness", "instrumentalness", "time_signature", "danceability",
          "key", "duration", "loudness", "valence", "mode"]

# Actual data structure for the data
myData = []

# Get first 100 songs here so we can get the total number of songs
playlist = sp.user_playlist_tracks(userId, playlistID, "",100,0)

totalNoSongs = playlist['total']
# Really smart way of rounding up to the nearest 100 and then adding 1 so the python range would include that hundred
if totalNoSongs % 100 != 0:
    totalNoSongs = ((totalNoSongs / 100) + 1) * 100 + 1
    
# Can only get 100 tracks at a time so we use an offset
for offset in range(100, totalNoSongs, 100):
    
    print("Getting songs from ", offset - 100, " to ", offset)
    
    # Add a delay because we are querying for a lot of data - don't want to flood
    time.sleep(10)
    
    for item in playlist['items'] :
        
        # Now for each song in the playlist we want to get certain features
        track = item['track']
        
        trackId = track['uri']
        songTitle = track['name']
        popularity = track['popularity']
        
        # A song might have more than one artist so we make a list of all of them
        artistName = []
        for artist in track['artists']:
            artistName.append(artist['name'])

        # Get features for the track
        features = sp.audio_features([trackId])
        
        # If the feature array is empty this usually means something has gone wrong 
        # with the request so this stops the program from failing in that case
        if features[0] != None :
            energy = features[0]['energy']
            liveness = features[0]['liveness'] 
            tempo = features[0]['tempo']
            speechiness = features[0]['speechiness']
            acousticness = features[0]['acousticness']
            instrumentalness = features[0]['instrumentalness']
            time_signature = features[0]['time_signature']
            danceability = features[0]['danceability']
            key = features[0]['key']
            duration_ms = features[0]['duration_ms']
            loudness = features[0]['loudness']
            valence = features[0]['valence']
            mode = features[0]['mode']
            
        # Create a new row of data for each song using the features above
        newRow = [trackId,songTitle, artistName, popularity, energy, liveness, tempo, speechiness, acousticness, instrumentalness, time_signature,
                  danceability, key, duration_ms, loudness, valence, mode]
        
        # Add the new row to our existing data
        myData.append(newRow)
        
        # Get the next 100 songs in the playlist - done at the end because we needed the total number of songs
        playlist = sp.user_playlist_tracks(userId, playlistID, "",100,offset)
        
print("DONE!")

# Store all the data in a DataFrame
df = pd.DataFrame(myData, columns=columns)

# Writing the data from the Spotify API to a CSV 
print("Writing file to CSV...")
df.to_csv(fileName, encoding="utf-8", header=True, index=False)

print("ALL DONE!")

# Read the data from the CSV to make sure everything is fine
data = pd.read_csv(fileName)
data.head()

## Feature description: 
![alt text](https://github.com/pawKer/predicting-music-popularity/blob/master/Images/features0.PNG?raw=true "Description of popularity")
![alt text](https://github.com/pawKer/predicting-music-popularity/blob/master/Images/features1.PNG?raw=true "Description of audio features part 1")
![alt text](https://github.com/pawKer/predicting-music-popularity/blob/master/Images/features2.PNG?raw=true "Description of audio features part 2")